# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore")

Посмотрим на наши данные

In [2]:
df = pd.read_csv('users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Перед нами данные пользователей двух тарифов: смарт - 0 и ультра - 1. Заметим, что значения разных параметров отличаются в сотни раз, что может плохо влиять на обучение модели.

## 2. Разбейте данные на выборки

Изначально у нас нет тестовой части для нашей задачи, сделаем её, взяв 20% данных.

In [4]:
X = df.drop(['is_ultra'], axis=1)
y = df['is_ultra']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

Не будем выделять отдельно часть для валидации, а применим кроссвалидацию, реализованную в библиотеке skikit-learn. Количество фолдов оставим равным 5.

## 3. Исследуйте модели

Рассмотрим 4 модели:
- случайный лес;
- логистическую регрессию;
- метод ближайщих соседей;
- метод опорных векторов.

В каждой модели сделаем нормализацию входных данных,а затем проведем решетчатый поиск оптимальных параметров. Для этого используем конвейер, реализованный в библиотеке skikit-learn, - pipeline, он позволяет объединять различные классы предварительной обработки в одну цепочку, а так же решает проблему просачивания информации из валидационной выборки на кроссвалидации при предварительной нормализации данных. 

Затем попробуем объединить все модели с помощью стэкинга и посмотрим, что получится.

#### Случайный лес

In [5]:
%%time
pipe = make_pipeline(MinMaxScaler(), RandomForestClassifier(random_state=42)) 
params = { 
    'randomforestclassifier__n_estimators': [100, 200, 500, 800, 1000],
    'randomforestclassifier__max_depth' : [4,5,6,7,8,10,15,20]
}
gridsearch_1 = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=2)
gridsearch_1.fit(X_train, y_train)
print('Best parameters:', gridsearch_1.best_params_)
print(f'Best mean score: {gridsearch_1.best_score_:.4}')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters: {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__n_estimators': 200}
Best mean score: 0.804
Wall time: 1min 20s


#### Логистическая регрессия

In [6]:
%%time
pipe = make_pipeline(MinMaxScaler(), LogisticRegression(random_state=42))

params = { 
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'logisticregression__penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'logisticregression__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
gridsearch_2 = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
gridsearch_2.fit(X_train, y_train)
print('Best parameters:', gridsearch_2.best_params_)
print(f'Best mean score: {gridsearch_2.best_score_:.4}')

Fitting 5 folds for each of 140 candidates, totalling 700 fits
Best parameters: {'logisticregression__C': 1000, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'liblinear'}
Best mean score: 0.7441
Wall time: 2.29 s


#### Метод ближайших соседей

In [7]:
%%time
pipe = make_pipeline(MinMaxScaler(), KNeighborsClassifier())

params = { 
    'kneighborsclassifier__n_neighbors': list(range(20)),
    'kneighborsclassifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}
gridsearch_3 = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
gridsearch_3.fit(X_train, y_train)
print('Best parameters:', gridsearch_3.best_params_)
print(f'Best mean score: {gridsearch_3.best_score_:.4}')

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__n_neighbors': 15}
Best mean score: 0.8001
Wall time: 3.67 s


#### Метод опорных векторов

In [8]:
%%time
pipe = make_pipeline(MinMaxScaler(), SVC(random_state=42))

params = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3, 1e-4], 'svc__C': [1, 10, 100, 1000]},
          {'svc__kernel': ['linear'], 'svc__C': [1, 10, 100, 1000]}]
gridsearch_4 = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
gridsearch_4.fit(X_train, y_train)
print('Best parameters:', gridsearch_4.best_params_)
print(f'Best mean score: {gridsearch_4.best_score_:.4}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'svc__C': 10, 'svc__kernel': 'linear'}
Best mean score: 0.7017
Wall time: 10.6 s


После обучения моделей получили следующее качество на валидационной выборке:
- случайный лес - 0.804
- логистическая регрессия - 0.744 
- метод ближайщих соседей - 0.801
- метод опорных векторов - 0.702

Только две модели прошли порог 75%. Лучше всего показал себя случайный лес, при том, что оптимизация шла только по двум параметрам.

#### Стэкинг

Данный метод позволяет использовать преимущества и сглаживать недостатки разных моделей. Используем все предыдущие модели с найденными оптимальными параметрами, в качестве итогового классификатора используем простую логистическую регрессию.

In [9]:
models = [
    ('randomforestclassifier', gridsearch_1.best_estimator_),
    ('logisticregression', gridsearch_2.best_estimator_),
    ('kneighborsclassifier', gridsearch_3.best_estimator_),
    ('svc', gridsearch_4.best_estimator_)
]

clf = StackingClassifier(estimators=models, final_estimator=LogisticRegression())
scores = cross_val_score(clf, X_train, y_train)
print(f'Best mean score: {scores.mean():.4}')

Best mean score: 0.8079


Получилось улучшить качество на 0.3%. Используем итоговую модель на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

In [10]:
clf.fit(X_train, y_train)
print(f'Score: {clf.score(X_test, y_test):.4}')

Score: 0.8149


Качество на тестовой выборке оказалось 0.815.

## 5. (бонус) Проверьте модели на адекватность

Посмотрим на влияние признаков в случайном лесе. 

In [11]:
for name, importance in zip(X.columns, gridsearch_1.best_estimator_.named_steps['randomforestclassifier'].feature_importances_):
    print(name, "=", importance)

calls = 0.2017688489406541
minutes = 0.2717015177916336
messages = 0.19877159732739008
mb_used = 0.32775803594032216


Больше всего на модель влияет интернет-трафик, что можно принять за правду, вспомнив, что больше всего переплат было как раз за мегабайты интернета. 

In [12]:
for name, importance in zip(X.columns, gridsearch_2.best_estimator_.named_steps['logisticregression'].coef_[0]):
    print(name, "=", importance)

calls = -0.7684954829703456
minutes = 3.3058062291925885
messages = 2.154313337188099
mb_used = 1.9949235041635025


In [13]:
for name, importance in zip(X.columns, gridsearch_4.best_estimator_.named_steps['svc'].coef_[0]):
    print(name, "=", importance)

calls = -0.0008427554296268625
minutes = 0.005041496380415289
messages = 0.004105726073843297
mb_used = 0.003696144459980566


Посмотрим на средние показатели данных, на которых модель ошибается.

In [14]:
# Ультра
test = X_test.copy()
test['true'] = y_test
test['predict'] = clf.predict(X_test)
test[(test['true']!=test['predict']) & (test['true']==1)].mean()

calls          55.449438
minutes       381.795730
messages       31.179775
mb_used     14612.776629
true            1.000000
predict         0.000000
dtype: float64

In [15]:
# Смарт
test[(test['true']!=test['predict']) & (test['true']==0)].mean()

calls          91.733333
minutes       660.856333
messages       51.066667
mb_used     21125.770000
true            0.000000
predict         1.000000
dtype: float64

Видим, что модель ошибается на клиентах тарифа ультра, расходующих сравнительно мало трафика, минут и смс, и на клиентах тарифа смарт, расходующих много трафика, минут и смс. Это нормально, бывают клиенты, которые не реализуют лимиты тарифов или, наоборот, сильно выходят за них.

In [16]:
print(classification_report(y_test, test['predict'], target_names=['smart', 'ultra']))

              precision    recall  f1-score   support

       smart       0.83      0.93      0.88       455
       ultra       0.77      0.53      0.62       188

    accuracy                           0.81       643
   macro avg       0.80      0.73      0.75       643
weighted avg       0.81      0.81      0.80       643



Почти половина пользователей тарифа ультра определяются моделью как пользователи тарифа смарт, возможно стоит подумать о рекомендациях клиентам.

## 6. Вывод

Мы разбили данные на тренировочную и тестовую выборки. Провели сравнительный анализ различных моделей для классификации, выбрали лучшую и проверили ее на тестовой выборке, получив качество 0.815. А также провели проверку модели на адекватность.